In [1]:
import json

from rlcache.backend import storage_from_config
from rlcache.cache_constants import CacheInformation, OperationType
from rlcache.observer import ObservationType
from rlcache.strategies.caching_strategies import caching_strategy_from_config
from rlcache.strategies.caching_strategies.rl_caching_strategy import CachingStrategyRLConverter
%load_ext autoreload
%autoreload 2

""" TODO later
DATABASE_BACKEND = storage_from_config(CONFIG['database_backend_settings'])

CACHE_BACKEND = storage_from_config(CONFIG['cache_backend_settings'])
CACHE_MANAGER = CacheManager(config=CONFIG['cache_manager_settings'],
                             cache=CACHE_BACKEND,
                             backend=DATABASE_BACKEND)
REQUESTS_COUNTER = Counter()
"""

" TODO later\nDATABASE_BACKEND = storage_from_config(CONFIG['database_backend_settings'])\n\nCACHE_BACKEND = storage_from_config(CONFIG['cache_backend_settings'])\nCACHE_MANAGER = CacheManager(config=CONFIG['cache_manager_settings'],\n                             cache=CACHE_BACKEND,\n                             backend=DATABASE_BACKEND)\nREQUESTS_COUNTER = Counter()\n"

In [2]:
config_file = 'configs/rl_caching_strategy.json'
with open(config_file, 'r') as fp:
    CONFIG = json.load(fp)

cache = storage_from_config(CONFIG['cache_backend_settings'])
shared_cache_info = CacheInformation(cache.capacity, cache.size)

caching__strategy_config = CONFIG['cache_manager_settings']['caching_strategy_settings']
caching_strategy = caching_strategy_from_config(caching__strategy_config, shared_cache_info)


19-04-07 19:32:32:INFO:Parsed state space definition: Floatbox((14,) <class 'numpy.float32'> )
19-04-07 19:32:32:INFO:Parsed action space definition: Intbox((1,) <class 'numpy.int32'> )
19-04-07 19:32:32:INFO:No preprocessing required.
19-04-07 19:32:32:INFO:Execution spec is: {'seed': 15, 'gpu_spec': {'gpus_enabled': True, 'max_usable_gpus': 1, 'allow_memory_growth': True}, 'disable_monitoring': True, 'session_config': {'allow_soft_placement': True, 'device_count': {'CPU': 1}, 'inter_op_parallelism_threads': 1, 'intra_op_parallelism_threads': 1, 'type': 'monitored-training-session', 'log_device_placement': False}, 'mode': 'single', 'distributed_spec': None, 'device_strategy': 'default', 'default_device': None, 'device_map': {}, 'enable_profiler': False, 'profiler_frequency': 1000, 'enable_timeline': False, 'timeline_frequency': 1}
19-04-07 19:32:32:INFO:GPU strategy initialized with GPUs enabled: ['/device:GPU:0']
19-04-07 19:32:32:WARNING:`device_map` given for device-strategy=`defau

/home/samialab/anaconda3/envs/rlcache/lib/python3.7/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


In [3]:
dummy_key = 'user6868534811834787757'
dummy_values =  {'field1': '2Ki"-2$>z\'087Vq.7d"+:)Ec1Iw7/`0Y5(#<%Fm"$:4%h6G\'\'0 !M=#"6=;j"C+86p;+>9Vy?L546z9@1=Ze3/`?^y0Vo7/|94n/',
                 'field0': '(Ei#Lw*4404n0>z22<"]5)8*$/b;Q345~2Aq\':8!6:=Q7-Z1((l76j;%>)Du?A7,#f/1t*&d=Ke#:";@#,]o,A}.!b9Ic;(>17r;',
                 'field7': '=/>0Pu:M59P)$(>$G9);j"Ik(&,(44=;>1*,7Ai&Y+(#6)Ri/6"996)R#8!b?Ms\'#4-2.= |-(v+24\'Ge3E!35x\'+4![{#Tu(%"9', 
                 'field6': ',(d&\\!"?>5),3%(%Xw?D-$(n4Rw\'%./:,!Ls:7$(Ay9949@3(.f"_m7Sq/B=4X{;:.(?b"L%==8<[i802 62 ^q"?t6%09D5;E\' ', 
                 'field9': '(344S)-_%4:"\'_a6&. ?n*Ks:.p.>$6+t\' 8)A+-Ho)((9Ig,,4,?h-\'b/@m/>x&E74]{6A+1> 09~5[u7F%>4`+?64J#=485Ym2',
                 'field8': '&Ls.Z3,H90S!(Gc7?"#F)-Q97&v\'-"/ t*@\x7f>-z/Qs >&,,2=X1=M9&=v#9*#)|7^c [!)\\7&\'j3!n3$t?Ay\'U!4Hy%-4%[7>L)%',
                 'field3': "/.`;%h>&j/K3-G{7V%1Uy>'&$\\-4E%8(t+F= ? )-d%\\\x7f6R)9H)$A')I\x7f+<23Di/I7>Yc>W{$B5)/>3*z:Je 2`'5r+Uq>^?&Rs#", 
                 'field2': '7B; ,|,#|>:40Yc*^y6"*51p\'1z7[#-X=-Su$Pc"Y\x7f\'.d;Ma8\\-- ~7+,9+t$Y{/Mg"3l.@u1J!$?"(Gy2Fa:0d.Si92,4M32Cu#',
                 'field5': '+*6 \\g"9d/:: !:!$26 8 A+60l2 v 4n#P\'7^)!J72021\'t##r>;,8De;X--2b2*z$)* >f%@c3<85Ay37z9Aq/R5*O7) b3Ku>', 
                 'field4': '(Ca2<:*Hg7O/26l>-27;0<T)8Wm?.6+2*1^}>+ 0No;Ss2Dg>3h2]q!+*1Ec1Xg.]3&3~.?0\'<.$Ru3)j3,.?Q1-L!#Py\'&,"$v&'}


In [4]:
# TODO the workload generator generates stupid values
caching_strategy.should_cache(dummy_key, dummy_values, 50, OperationType.New)

True

In [5]:
for i in range(150):
    caching_strategy.observe(dummy_key, ObservationType.Hit, {})

reward: 1
reward: 2
reward: 3
reward: 4
reward: 5
reward: 6
reward: 7
reward: 8
reward: 9
reward: 10
reward: 11
reward: 12
reward: 13
reward: 14
reward: 15
reward: 16
reward: 17
reward: 18
reward: 19
reward: 20
reward: 21
reward: 22
reward: 23
reward: 24
reward: 25
reward: 26
reward: 27
reward: 28
reward: 29
reward: 30
reward: 31
reward: 32
reward: 33
reward: 34
reward: 35
reward: 36
reward: 37
reward: 38
reward: 39
reward: 40
reward: 41
reward: 42
reward: 43
reward: 44
reward: 45
reward: 46
reward: 47
reward: 48
reward: 49
reward: 50
reward: 51
reward: 52
reward: 53
reward: 54
reward: 55
reward: 56
reward: 57
reward: 58
reward: 59
reward: 60
reward: 61
reward: 62
reward: 63
reward: 64
reward: 65
reward: 66
reward: 67
reward: 68
reward: 69
reward: 70
reward: 71
reward: 72
reward: 73
reward: 74
reward: 75
reward: 76
reward: 77
reward: 78
reward: 79
reward: 80
reward: 81
reward: 82
reward: 83
reward: 84
reward: 85
reward: 86
reward: 87
reward: 88
reward: 89
reward: 90
reward: 91
reward: 

In [ ]:
conv = CachingStrategyRLConverter(10)
conv.system_to_agent_reward(caching_strategy._incomplete_experience_storage.get(dummy_key)
)

In [7]:
caching_strategy._incomplete_experience_storage.get(dummy_key)

state: [ 2 13  0  0  3  4  5  6  7  8  9 10 11 12], agent_action: [1], ttl: 50, hits: 150, miss: 0

In [8]:
caching_strategy.observe(dummy_key, ObservationType.Invalidate, {})

Terminal state
reward: 150


In [9]:

caching_strategy.observe(dummy_key, ObservationType.Miss, {})

In [10]:

for episode in range(30000):
    should = caching_strategy.should_cache(dummy_key, dummy_values, 50, OperationType.New)
    if should:
        for i in range(300):
            caching_strategy.observe(dummy_key, ObservationType.Hit, {})
        caching_strategy.observe(dummy_key, ObservationType.Invalidate, {})
    else:
        caching_strategy.observe(dummy_key, ObservationType.Miss, {})

Terminal state
reward: -1
Terminal state
reward: -1
reward: 1
reward: 2
reward: 3
reward: 4
reward: 5
reward: 6
reward: 7
reward: 8
reward: 9
reward: 10
reward: 11
reward: 12
reward: 13
reward: 14
reward: 15
reward: 16
reward: 17
reward: 18
reward: 19
reward: 20
reward: 21
reward: 22
reward: 23
reward: 24
reward: 25
reward: 26
reward: 27
reward: 28
reward: 29
reward: 30
reward: 31
reward: 32
reward: 33
reward: 34
reward: 35
reward: 36
reward: 37
reward: 38
reward: 39
reward: 40
reward: 41
reward: 42
reward: 43
reward: 44
reward: 45
reward: 46
reward: 47
reward: 48
reward: 49
reward: 50
reward: 51
reward: 52
reward: 53
reward: 54
reward: 55
reward: 56
reward: 57
reward: 58
reward: 59
reward: 60
reward: 61
reward: 62
reward: 63
reward: 64
reward: 65
reward: 66
reward: 67
reward: 68
reward: 69
reward: 70
reward: 71
reward: 72
reward: 73
reward: 74
reward: 75
reward: 76
reward: 77
reward: 78
reward: 79
reward: 80
reward: 81
reward: 82
reward: 83
reward: 84
reward: 85
reward: 86
reward: 87


KeyboardInterrupt: 